In [1]:
from sklearn.externals import joblib
import dill

classicML = False

print("\nStarting evaluation..")
print("--------------------------------------\n")
print("Loading pipeline")
pipeline = joblib.load("pipeline.pkl")
print("pipeline loaded")

if classicML == False:
    from keras.models import load_model
    print("Loading model")
    classifier = load_model("model.h5")
    print("Model loaded")


Starting evaluation..
--------------------------------------

Loading pipeline
pipeline loaded


Using TensorFlow backend.


Loading model
Model loaded


Get evaluation data

In [7]:
import pandas as pd
test_df = pd.read_csv("test_data.csv")
X = test_df.iloc[:,1:]
y = test_df.iloc[:,0]

print("Running predictions")
if classicML == False:
    X = pipeline.transform(X)
    y_pred = classifier.predict_classes(X)
else:
    y_pred = pipeline.predict(X)



Running predictions


In [8]:
y_pred

array([0, 7, 7, ..., 7, 6, 9])

In [9]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
cm = confusion_matrix(y, y_pred)
accuracy = accuracy_score(y, y_pred)

print("Test results\n-------------------------------------------------------------\n")
results = {"Accuracy":accuracy, "Confusion matrix":cm.tolist()} #TODO: Also include used train and test size
print("Accuracy: {0}%\n".format(accuracy))
print("Confusion matrix:")
print(cm)

Test results
-------------------------------------------------------------

Accuracy: 0.9595238095238096%

Confusion matrix:
[[833   0   2   2   1   1   7   1   4   1]
 [  0 937   2   1   2   1   1   1   3   2]
 [  9   0 767   4   6   1   4   6   2   2]
 [  4   6   8 830   0  11   2   7  10   4]
 [  4   4   4   0 770   1   4   2   2  20]
 [  8   2   1  11   5 698   8   2   6   6]
 [  6   3   4   0   4   6 807   0   1   0]
 [  2   2   4   3   0   2   0 864   2   5]
 [  2  11   4   2   4   7   3   0 767   2]
 [  6   2   2   9  15   3   0  10   6 787]]


In [4]:
import json

foundResults = False

from pathlib import Path
if Path('results.json').exists():
    with open('results.json', 'r') as f:
        previousResults = json.load(f)
    foundResults = True
else:
    print("Warning, did not find any previous results!!")

In [5]:
def compareResults(Results, PreviousResults):
    print("Comparing results")
    if(Results['Accuracy'] >= PreviousResults['Accuracy']):
        print("\nModel passed the test.")
        return True
    else:
        print("Model did not pass the test.")
        return False
    
def modelPassed():
    #Save the results as the current model
    with open('results.json', 'w') as f:
        json.dump(results, f, ensure_ascii=False, sort_keys=True, indent = 4)
    #Save the results in version control
    with open('./Versions/Pipeline_' + version +'_results.json', 'w') as f:
        json.dump(results, f, ensure_ascii=False, sort_keys=True, indent = 4)

def modelFailed():  
    print("Saving results..")
    with open('./Versions/Failed/Pipeline_' + version +'_results.json', 'w') as f:
        json.dump(results, f, ensure_ascii=False, sort_keys=True, indent = 4)
    print("Results saved.")
    import sys
    sys.exit(1)

In [6]:
import datetime
version = str(datetime.datetime.now())

if foundResults:
    if compareResults(results, previousResults):
        modelPassed()
    else:
        modelFailed()
else:
    print("Deploying model without previous results.")
    modelPassed()

Comparing results

Model passed the test.
